In [ ]:
import numpy as np
import trimesh
import pickle
from trimesh.viewer import SceneViewer
import pyglet
import glob
import joblib
import os

In [ ]:
class SMPLModel():
    def __init__(self, model_path):
        with open(model_path, 'rb') as f:

            params = pickle.load(f, encoding='latin1')
            self.J_regressor = params['J_regressor']
            self.weights = params['weights']
            self.posedirs = params['posedirs']
            self.v_template = params['v_template']
            self.shapedirs = params['shapedirs']
            self.faces = params['f']
            self.kintree_table = params['kintree_table']

        id_to_col = {
            self.kintree_table[1, i]: i for i in range(self.kintree_table.shape[1])
        }
        self.parent = {
            i: id_to_col[self.kintree_table[0, i]]
            for i in range(1, self.kintree_table.shape[1])
        }

        self.pose_shape = [24, 3]
        self.beta_shape = [10]
        self.trans_shape = [3]

        self.pose = np.zeros(self.pose_shape)
        self.beta = np.zeros(self.beta_shape)
        self.trans = np.zeros(self.trans_shape)

        self.verts = None
        self.J = None
        self.R = None

        self.update()

    def set_params(self, pose=None, beta=None, trans=None):
        if pose is not None:
            self.pose = pose
        if beta is not None:
            self.beta = beta
        if trans is not None:
            self.trans = trans
        self.update()
        return self.verts

    def update(self):
        # how beta affect body shape
        v_shaped = self.shapedirs.dot(self.beta) + self.v_template

        # joints location due to changed body shape
        self.J = self.J_regressor.dot(v_shaped)
        pose_cube = self.pose.reshape((-1, 1, 3))

        # rotation matrix for each joint
        self.R = self.rodrigues(pose_cube)
        I_cube = np.broadcast_to(np.expand_dims(np.eye(3), axis=0), (self.R.shape[0] - 1, 3, 3))
        lrotmin = (self.R[1:] - I_cube).ravel()

        # how pose affect body shape in zero pose
        v_posed = v_shaped + self.posedirs.dot(lrotmin)

        # world transformation of each joint
        G = np.empty((self.kintree_table.shape[1], 4, 4))
        G[0] = self.with_zeros(np.hstack((self.R[0], self.J[0, :].reshape([3, 1]))))
        for i in range(1, self.kintree_table.shape[1]):
            G[i] = G[self.parent[i]].dot(self.with_zeros(np.hstack([self.R[i], ((self.J[i, :] - self.J[self.parent[i], :]).reshape([3, 1]))])))

        # remove the transformation due to the rest pose
        G = G - self.pack(np.matmul(G,np.hstack([self.J, np.zeros([24, 1])]).reshape([24, 4, 1])))

        # transformed vertices
        T = np.tensordot(self.weights, G, axes=[[1], [0]])
        rest_shape_h = np.hstack((v_posed, np.ones([v_posed.shape[0], 1])))
        v = np.matmul(T, rest_shape_h.reshape([-1, 4, 1])).reshape([-1, 4])[:, :3]
        self.verts = v + self.trans.reshape([1, 3])

        # transformed joints
        self.J = self.J_regressor.dot(self.verts)


    def rodrigues(self, r):
        theta = np.linalg.norm(r, axis=(1, 2), keepdims=True)
        # avoid zero divide
        theta = np.maximum(theta, np.finfo(np.float64).tiny)
        r_hat = r / theta
        cos = np.cos(theta)
        z_stick = np.zeros(theta.shape[0])

        m = np.dstack([z_stick, -r_hat[:, 0, 2], r_hat[:, 0, 1], r_hat[:, 0, 2], z_stick, -r_hat[:, 0, 0], -r_hat[:, 0, 1], r_hat[:, 0, 0], z_stick]).reshape([-1, 3, 3])

        i_cube = np.broadcast_to(np.expand_dims(np.eye(3), axis=0), [theta.shape[0], 3, 3])

        A = np.transpose(r_hat, axes=[0, 2, 1])
        B = r_hat
        dot = np.matmul(A, B)
        R = cos * i_cube + (1 - cos) * dot + np.sin(theta) * m

        return R

    def with_zeros(self, x):
        return np.vstack((x, np.array([[0.0, 0.0, 0.0, 1.0]])))

    def pack(self, x):
        return np.dstack((np.zeros((x.shape[0], 4, 3)), x))

## Displaying the fitted SMPL mesh

In [ ]:
smpl = SMPLModel('SMPLmodel/models/smpl/SMPL_NEUTRAL.pkl')
dataset = 'test'
frame = 0

# render mesh
params = joblib.load(f'Rendered/{dataset}.pkl')
smpl.pose = params['pose'][frame]
smpl.beta = params['beta']
smpl.trans = params['trans'][frame]
smpl.update()
mesh = trimesh.Trimesh(smpl.verts, smpl.faces, face_colors=[0.65,0.74,0.86,1])

# render joints
joints3d = np.load(f'Results/{dataset}.npy')[frame]
p = trimesh.points.PointCloud(joints3d, colors=[0,0,255,255])

scene = trimesh.Scene([mesh,p])
scene.set_camera(distance=3, center=[0,0,0], resolution=[720, 720])

scene.show()

# png = scene.save_image(resolution=[720, 720], visible=True)
# with open('test_mesh.png', 'wb') as f:
#     f.write(png)
#     f.close()

## Saving the rendered mesh as images and video

In [ ]:
def color(dataset):
    """Define different colors for different methods"""
    if dataset.find('Method_1') == 0:
        return [0,0,0,1]
    elif dataset.find('Method_2') == 0:
        return [1,1,0,1]
    elif dataset.find('Method_3') == 0:
        return [1,0,1,1]
    elif dataset.find('Method_4') == 0:
        return [1,0,0,1]
    elif dataset.find('Method_5') == 0:
        return [0,1,0,1]
    elif dataset.find('Method_6') == 0:
        return [0,0,1,1]
    else:
        return [0.65,0.74,0.86,1]

In [ ]:
class MeshWindow():
    counter = 0
    def __init__(self, dataset, fps=25):
        """
        dataset is the dataset name in Rendered/{dataset}.pkl
        """
        
        self.dataset = dataset
        self.colors = color(dataset)
        self.fps = fps
        
        # Initialise the SMPL model
        self.smpl = SMPLModel('SMPLmodel/models/smpl/SMPL_NEUTRAL.pkl')
        
        self.output_dir = f'Video/{dataset}'
        if not os.path.isdir(self.output_dir):
            os.makedirs(self.output_dir, exist_ok=True)
        
        # Load SMPL parameters
        params = joblib.load(f'Rendered/{dataset}.pkl')
        self.pose_params = params['pose']
        self.nframes = len(self.pose_params)
                
        mesh = trimesh.Trimesh(self.smpl.verts, self.smpl.faces, face_colors=[0,0,0,0])
        self.scene = trimesh.Scene(mesh)
        self.scene.set_camera(distance=3, center=[0,0,0], resolution=[1920, 1080])
        
    def update(self, scene):
        self.smpl.pose = self.pose_params[MeshWindow.counter]
#         self.smpl.trans = self.trans_params[MeshWindow.counter]
        self.smpl.update()
        scene.geometry.clear()
        scene.add_geometry(trimesh.Trimesh(self.smpl.verts, self.smpl.faces, face_colors=self.colors))
        
        MeshWindow.counter = (MeshWindow.counter+1) % self.nframes
        
    def save_frames(self):
        try:
            lastframe = int(glob.glob(f'{self.output_dir}/{self.dataset}/*.png')[-1][-7:-4])
        except:
            lastframe = 0
        for i in range(lastframe, len(self.pose_params)):
            self.smpl.pose = self.pose_params[i]
#             self.smpl.trans = self.trans_params[i]
            self.smpl.update()
            while True:
                try:
                    self.scene.geometry.clear()
                    self.scene.add_geometry(trimesh.Trimesh(self.smpl.verts, self.smpl.faces, face_colors=self.colors))

                    file_name = '{}/{:03d}.png'.format(self.output_dir, i)
                    # save a render of the object as a png
                    png = self.scene.save_image(resolution=[1920, 1080], visible=True)
                    with open(file_name, 'wb') as f:
                        f.write(png)
                        f.close()
                except:
                    continue
                break
                
    def make_video(self):
        os.system('ffmpeg -y -i {}/%03d.png -c:v libx264 -vf "fps={},format=yuv420p" {}/Video.mp4'.format(self.output_dir, self.fps, self.output_dir))
    
    def show(self):
        SceneViewer(self.scene, callback=self.update)

In [ ]:
mymesh = MeshWindow(dataset='test', fps=15)
mymesh.save_frames()
# mymesh.make_video()

In [ ]:
mymesh.show()